This notebook shows how to use our openai logging integration to monitor api calls

In [ ]:
import weave
weave.use_frontend_devmode()
from weave.legacy.weave.monitoring import openai, init_monitor

OPENAI_MODEL = 'gpt-3.5-turbo'

In [ ]:
# Pass <wandb_team_or_user>/<wandb_project>/<table_name>
m = init_monitor('shawn/oai-mon/test21')

# Do an initial request, otherwise we don't have a type on which to recommend the OpenAI board!
# We need at least 2 requests for the Board to work, otherwise we get divide by zero errors.
# TODO: fix this onboarding issue
r = openai.ChatCompletion.create(model=OPENAI_MODEL, messages=[{"role": "user", "content": f"hello world!"}])
r = openai.ChatCompletion.create(model=OPENAI_MODEL, messages=[{"role": "user", "content": f"what is 2+2?"}])

Click the link above to go to the Weave UI for the table we're logging to.

From there you can click "OpenAI Monitor Board" to create a Weave Board for this data stream.

In [ ]:
# Monitor ChatCompletion requests
r = openai.ChatCompletion.create(model=OPENAI_MODEL, messages=[
        {"role": "user", "content": f"who won the world series in 2006?"},
    ])
r['choices'][0]['message']['content']

In [ ]:
# To keep track of prompts and parameters, add them to attributes on the logged
# record.

system_prompt = "you always write in bullet points"
prompt = 'solve the following equation step by step: {equation}'
params = {'equation': '4 * (3 - 1)'}
openai.ChatCompletion.create(model=OPENAI_MODEL,
                             messages=[
                                    {"role": "system", "content": system_prompt},
                                    {"role": "user", "content": prompt.format(**params)},
                                ],
                             # you can add additional attributes to the logged record
                             # see the monitor_api notebook for more examples
                             monitor_attributes={
                                 'system_prompt': system_prompt,
                                 'prompt': prompt,
                                 'params': params
                             })

In [ ]:
# Monitor streaming requests
# TODO: we don't get token counts here yet.
from weave.legacy.weave.monitoring.openai import message_from_stream
r = openai.ChatCompletion.create(model=OPENAI_MODEL, messages=[
        {"role": "system", "content": "Your are a robot and only speak in robot, like beep bloop bop."},
        {"role": "user", "content": f"Tell me a 50 word story."},
    ], stream=True)
for s in message_from_stream(r):
    print(s, end='')

In [ ]:
# Render table inline
#m.rows()

In [ ]:
# Render board inline
# from weave.legacy.weave.panels_py import panel_llm_monitor
# board = panel_llm_monitor.board.raw_resolve_fn(m.rows())
# board

In [ ]:
# Other examples

# TODO: restore async variant

# result = await monitored_a_create(model="gpt-3.5-turbo", messages=[
#         {"role": "system", "content": "You are a world-class machine learning researcher."},
#         {"role": "user", "content": f"Please provide a simple, fact-based question to send to an AI system. Do not say anything other than the question itself. Use this random number as inspiration: {random.random()}."},
#     ])
# result

# TODO: ensure works with function calls
# functions = [
#     {
#         "name": "get_current_weather",
#         "description": "Get the current weather",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "location": {
#                     "type": "string",
#                     "description": "The city and state, e.g. San Francisco, CA",
#                 },
#                 "format": {
#                     "type": "string",
#                     "enum": ["celsius", "fahrenheit"],
#                     "description": "The temperature unit to use. Infer this from the users location.",
#                 },
#             },
#             "required": ["location", "format"],
#         },
#     },
#     {
#         "name": "get_n_day_weather_forecast",
#         "description": "Get an N-day weather forecast",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "location": {
#                     "type": "string",
#                     "description": "The city and state, e.g. San Francisco, CA",
#                 },
#                 "format": {
#                     "type": "string",
#                     "enum": ["celsius", "fahrenheit"],
#                     "description": "The temperature unit to use. Infer this from the users location.",
#                 },
#                 "num_days": {
#                     "type": "integer",
#                     "description": "The number of days to forecast",
#                 }
#             },
#             "required": ["location", "format", "num_days"]
#         },
#     },
# ]

# result = openai.ChatCompletion.create(model="gpt-3.5-turbo", functions=functions, messages=[
#         {"role": "system", "content": "You love to call functions."},
#         {"role": "user", "content": f"what's the weather today"},
#     ])